In [1]:

import awswrangler  as wr
import pandas as pd
import pymysql
import datetime
import requests
import json

# def lambda_handler(event, context):
sql = """
with log_raw as (
select *
, case when user_agent like '%iPhone%' then 1 else 0 end as is_iphone
from moyo_logs.user_behavior_logs
where 1=1
--     and date_ymd >= '2023-09-09'
    and date_ymd >= '2023-06-10'
    and date_ymd < '2023-09-12' -- last 90일
-- limit 100
)
, iphone as (
select
distinct user_id
from log_raw
where 1=1
    and is_iphone = 1
)
, User_agg as (
select
user_id
, min( client_access_time) as first_time
, max( client_access_time) as last_time
, count(*) as cnt
, count( case when category = 'apply'
            and object_type = 'plan' then 1 end ) as apply_cnt
from log_raw
where 1=1
group by 1
)
, rel_page as (
select
distinct user_id
from log_raw a
where 1=1
    and (a.navigation = 'search_phones'
        or    a.navigation = 'phone_contents'
        or  a.navigation = 'contents_detail'
        )
)
, iphone15 as (
select
distinct  user_id
from moyo_data."signup_event_meta"
where 1=1
    and event_type = 'pre_iphone15_register'
group by 1
)
, stock as (
select
distinct user_id
from moyo_logs."amplitude_logs"
where 1=1
    -- and date_ymd  = '2022-06-09'
    and event_type =  'Click_알림받기_재고알리미'
-- group by 1 -- 명목 < 실질
    and user_id is not null
-- group by 1
)
, raw as (
select a.*
, d.profile
, d.phone_number
,  REPLACE(REPLACE(REPLACE(d.phone_number, '+82 ', '0'), '-', ''), ' ', '') as formatted_phone
, case when b.user_id is not null then 1 else 0 end as is_iphone
, case when c.user_id is not null then 1 else 0 end as is_iphone15
, case when e.user_id is not null then 1 else 0 end as is_rel_page
, e.user_id as stock_user_id
, case when f.user_id is not null then 1 else 0 end as is_stock
, marketing_consent
from user_agg  a
join moyo_data.moyo_user_v2 d on d.id = a.user_id
left join iphone b on a.user_id = b.user_id
left join iphone15 c on a.user_id = c.user_id
left join rel_page e on a.user_id = e.user_id
full outer join stock f on a.user_id = cast ( f.user_id as integer )
where 1=1
     and d.marketing_consent in ( 1, 0 )
)
select
coalesce(user_id ,stock_user_id ) as user_id
, phone_number
, formatted_phone
, profile
, first_time
, last_time
, cnt
, apply_cnt
, is_iphone
,  is_iphone15
, is_rel_page
, is_stock
, marketing_consent
from raw
where 1=1
--   and user_id is null

"""

df = wr.athena.read_sql_query(sql=sql, database="moyo_order")
# bucket = 'moyo-mart'
# table_name = 'paid_ads_card'

# res = wr.s3.to_parquet(
#     df=df,
#     path=f"s3://{bucket}/{table_name}/",
#     dataset=True,
#     database="moyo_mart",
#     table=table_name, 
#     mode="overwrite", 
#     partition_cols=['date_ymd']
# )


# # https://s3.console.aws.amazon.com/s3/buckets/moyo-mart?region=ap-northeast-2&prefix=ringo_partition_test/&showversions=false
# sql_query = """

# """

# df = wr.athena.read_sql_query(sql=sql_query, database="moyo_logs")

# bucket = 'moyo-mart'
# table_name = 'huchat_crm_230912'

# res = wr.s3.to_parquet(
#     df=df,
#     path=f"s3://{bucket}/{table_name}/",
#     dataset=True,
#     database="moyo_mart",
#     table=table_name,
#     mode="overwrite_partitions",
#     partition_cols=['append']
# )

In [2]:
import numpy as np
np.random.seed(42)

df['ab'] = np.random.choice(['a', 'b'], size=len(df))


In [3]:
bucket = 'moyo-mart'
table_name = 'huchat_crm_230912'

res = wr.s3.to_parquet(
    df=df,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite",
    # partition_cols=['append']
)

In [4]:

import awswrangler  as wr
import pandas as pd
import pymysql
import datetime
import requests
import json

# def lambda_handler(event, context):
sql = """
select
a.id as user_id 
, REPLACE(REPLACE(REPLACE(b.phone_number, '+82 ', '0'), '-', ''), ' ', '') as formatted_phone
from moyo_data."signup_event_meta" a
join moyo_data.moyo_user_v2 b on b.id = a.user_id
where 1=1
    and event_type = 'pre_iphone15_register'
    and b.phone_number is not null

"""

df2 = wr.athena.read_sql_query(sql=sql, database="moyo_order")
# bucket = 'moyo-mart'
# table_name = 'paid_ads_card'

# res = wr.s3.to_parquet(
#     df=df,
#     path=f"s3://{bucket}/{table_name}/",
#     dataset=True,
#     database="moyo_mart",
#     table=table_name, 
#     mode="overwrite", 
#     partition_cols=['date_ymd']
# )


# # https://s3.console.aws.amazon.com/s3/buckets/moyo-mart?region=ap-northeast-2&prefix=ringo_partition_test/&showversions=false
# sql_query = """

# """

# df = wr.athena.read_sql_query(sql=sql_query, database="moyo_logs")

# bucket = 'moyo-mart'
# table_name = 'huchat_crm_230912'

# res = wr.s3.to_parquet(
#     df=df,
#     path=f"s3://{bucket}/{table_name}/",
#     dataset=True,
#     database="moyo_mart",
#     table=table_name,
#     mode="overwrite_partitions",
#     partition_cols=['append']
# )

In [ ]:
bucket = 'moyo-mart'
table_name = 'huchat_crm_230912_request'

res = wr.s3.to_parquet(
    df=df2,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite",
    # partition_cols=['append']
)